In [10]:
import swe_bench_constants
from test_spec import make_test_spec, get_test_specs_from_dataset, TestSpec
from datasets import load_dataset
import subprocess
import os
dataset = load_dataset('princeton-nlp/SWE-bench_Lite', split='test')
def get_spec(sbi: swe_bench_constants.SWEbenchInstance) -> swe_bench_constants.SpecInstance:
    return swe_bench_constants.MAP_REPO_VERSION_TO_SPECS[sbi['repo']][sbi['version']]
def get_data(i: int) -> tuple[swe_bench_constants.SWEbenchInstance, swe_bench_constants.SpecInstance]:
    return dataset[i], get_spec(dataset[i])

In [11]:
get_data(10)

({'repo': 'django/django',
  'instance_id': 'django__django-11039',
  'base_commit': 'd5276398046ce4a102776a1e67dcac2884d80dfe',
  'patch': 'diff --git a/django/core/management/commands/sqlmigrate.py b/django/core/management/commands/sqlmigrate.py\n--- a/django/core/management/commands/sqlmigrate.py\n+++ b/django/core/management/commands/sqlmigrate.py\n@@ -55,8 +55,9 @@ def handle(self, *args, **options):\n                 migration_name, app_label))\n         targets = [(app_label, migration.name)]\n \n-        # Show begin/end around output only for atomic migrations\n-        self.output_transaction = migration.atomic\n+        # Show begin/end around output for atomic migrations, if the database\n+        # supports transactional DDL.\n+        self.output_transaction = migration.atomic and connection.features.can_rollback_ddl\n \n         # Make a plan that represents just the requested migrations and show SQL\n         # for it\n',
  'test_patch': 'diff --git a/tests/migrations/t

In [12]:
d_specs = get_test_specs_from_dataset("./foo", dataset)

In [13]:
d_specs

[TestSpec(instance_id='astropy__astropy-12907', repo='astropy/astropy', version='4.3', repo_script_list=['git clone -o origin https://github.com/astropy/astropy ./foo/astropy__astropy-12907', 'chmod -R 777 ./foo/astropy__astropy-12907', 'cd ./foo/astropy__astropy-12907', 'git reset --hard d16bfe05a744909de4b27f5875fe0d4ed41ce607', 'git remote remove origin', 'source /opt/miniconda3/bin/activate', 'conda activate astropy__astropy-12907', 'echo "Current environment: $CONDA_DEFAULT_ENV"', 'sed -i \'s/requires = \\["setuptools",/requires = \\["setuptools==68.0.0",/\' pyproject.toml', 'python -m pip install -e .[test] --verbose'], eval_script_list=['source /opt/miniconda3/bin/activate', 'conda activate astropy__astropy-12907', 'cd ./foo/astropy__astropy-12907', 'git config --global --add safe.directory ./foo/astropy__astropy-12907', 'cd ./foo/astropy__astropy-12907', 'git status', 'git show', 'git diff d16bfe05a744909de4b27f5875fe0d4ed41ce607', 'source /opt/miniconda3/bin/activate', 'conda 

In [14]:
d = d_specs[10]

In [15]:
for l in d.repo_script_list:
    print(l)

git clone -o origin https://github.com/django/django ./foo/django__django-11039
chmod -R 777 ./foo/django__django-11039
cd ./foo/django__django-11039
git reset --hard d5276398046ce4a102776a1e67dcac2884d80dfe
git remote remove origin
source /opt/miniconda3/bin/activate
conda activate django__django-11039
echo "Current environment: $CONDA_DEFAULT_ENV"
python -m pip install -e .


In [16]:
for l in d.eval_script_list:
    print(l)

source /opt/miniconda3/bin/activate
conda activate django__django-11039
cd ./foo/django__django-11039
sed -i '/en_US.UTF-8/s/^# //g' /etc/locale.gen && locale-gen
export LANG=en_US.UTF-8
export LANGUAGE=en_US:en
export LC_ALL=en_US.UTF-8
git config --global --add safe.directory ./foo/django__django-11039
cd ./foo/django__django-11039
git status
git show
git diff d5276398046ce4a102776a1e67dcac2884d80dfe
source /opt/miniconda3/bin/activate
conda activate django__django-11039
python -m pip install -e .
git checkout d5276398046ce4a102776a1e67dcac2884d80dfe tests/migrations/test_commands.py
git apply -v - <<'EOF_114329324912'
diff --git a/tests/migrations/test_commands.py b/tests/migrations/test_commands.py
--- a/tests/migrations/test_commands.py
+++ b/tests/migrations/test_commands.py
@@ -536,7 +536,13 @@ def test_sqlmigrate_forwards(self):
         index_op_desc_unique_together = output.find('-- alter unique_together')
         index_tx_end = output.find(connection.ops.end_transaction_sql

In [17]:
for l in d.env_script_list:
    print(l)

source /opt/miniconda3/bin/activate
conda create -n django__django-11039 python=3.6 -y
cat <<'EOF_59812759871' > $HOME/requirements.txt
asgiref ~= 3.2
argon2-cffi >= 16.1.0
bcrypt
docutils
geoip2
jinja2 >= 2.9.2
numpy
Pillow != 5.4.0
pylibmc; sys.platform != 'win32'
python-memcached >= 1.59
pytz
pywatchman; sys.platform != 'win32'
PyYAML
selenium
sqlparse >= 0.2.2
tblib >= 1.5.0

EOF_59812759871
conda activate django__django-11039 && python -m pip install -r $HOME/requirements.txt
rm $HOME/requirements.txt
conda activate django__django-11039


In [ ]:
def setup_testspec(ts: TestSpec):
    # execute each command in env_script_list
    for l in ts.env_script_list:
        subprocess.run(l, shell=True)
    # execute each command in repo_script_list
    for l in ts.repo_script_list:
        subprocess.run(l, shell=True)